In [1]:
import os
import pandas as pd

In [2]:
folder_path = 'Forecasts'  # Replace with your folder path
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]


In [3]:
districts = ['AHMEDNAGAR', 'AKOLA', 'AMRAVATI', 'AURANGABAD', 'BEED',
       'BHANDARA', 'BULDHANA', 'CHANDRAPUR', 'DHULE', 'GADCHIROLI',
       'GONDIA', 'HINGOLI', 'JALGAON', 'JALNA', 'KOLHAPUR', 'LATUR',
       'MUMBAI', 'NAGPUR', 'NANDED', 'NANDURBAR', 'NASHIK', 'OSMANABAD',
       'PALGHAR', 'PARBHANI', 'PUNE', 'RAIGAD', 'RATNAGIRI', 'SANGLI',
       'SATARA', 'SINDHUDURG', 'SOLAPUR', 'THANE', 'WARDHA', 'WASHIM',
       'YAVATMAL', 'MUMBAI SUBURBAN']

In [4]:
dfs = []  # list to collect all df_req

for f in csv_files:
    df = pd.read_csv(f"{folder_path}/{f}")
    model = f.split('_')[0]
    
    try:
        df_req = df[['district', 'date', 'actual', 'forecast']]
    except KeyError:
        if model == 'bayesian':
            df_req = df[['district', 'date', 'actual', 'forecast_mean']]
            df_req = df_req.rename(columns={'forecast_mean': 'forecast'})  # optional: standardize column name
    df_req['model'] = model 
    dfs.append(df_req)  # collect the processed df_req

# After the loop, concatenate all collected dfs
final_df = pd.concat(dfs, ignore_index=True)

            
        
    # print(df_req.shape)

C:\Users\nauti\AppData\Local\Temp\ipykernel_41336\2311738704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['model'] = model
C:\Users\nauti\AppData\Local\Temp\ipykernel_41336\2311738704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['model'] = model


In [5]:
final_df.columns

Index(['district', 'date', 'actual', 'forecast', 'model'], dtype='object')

In [6]:
final_df = final_df[['model','district', 'date', 'actual', 'forecast']]

In [7]:
final_df['model'] = final_df['model'].str.upper()  # Convert district names to uppercase

In [8]:
final_df.columns

Index(['model', 'district', 'date', 'actual', 'forecast'], dtype='object')

In [9]:
import numpy as np
import pandas as pd

# Assuming 'final_df' already loaded, with columns: ['district', 'date', 'actual', 'forecast', 'model']

# Empty list to collect results
results = []

districts = final_df['district'].unique()
models = final_df['model'].unique()

for district in districts:
    for model in models:
        temp_df = final_df[(final_df['district'] == district) & (final_df['model'] == model)]
        if not temp_df.empty:
            rmse = np.sqrt(((temp_df['forecast'] - temp_df['actual'])**2).mean())
            results.append({
                'district': district,
                'model': model,
                'RMSE': rmse
            })

# Create DataFrame
rmse_df = pd.DataFrame(results)

# Sort if you want
rmse_df = rmse_df.sort_values(by=['district', 'RMSE'])

# See top results
print(rmse_df.head())


      district       model       RMSE
1   AHMEDNAGAR       ARIMA  15.129452
17  AHMEDNAGAR       TBATS  15.505437
6   AHMEDNAGAR         EXP  15.746770
2   AHMEDNAGAR    BAYESIAN  15.812603
4   AHMEDNAGAR  ELASTICNET  16.187799


In [10]:
rmse_df[rmse_df.district == "THANE"].sort_values(by='RMSE', ascending=True)

,district,model,RMSE
644,THANE,RF,10.941329
648,THANE,XGBOOST,11.472098
638,THANE,GB,11.933597
640,THANE,KNN,13.508375
633,THANE,BAYESIAN,14.221095
646,THANE,SARIMA,14.806250
636,THANE,ET,16.418379
635,THANE,ELASTICNET,17.216445
642,THANE,LINEAR,17.322603
641,THANE,LASSO,17.705485


In [11]:
final_df['model'].unique()

array(['ANN', 'ARIMA', 'BAYESIAN', 'DT', 'ELASTICNET', 'ET', 'EXP', 'GB',
       'HUBER', 'KNN', 'LASSO', 'LINEAR', 'LSTM', 'PROPHET', 'RF',
       'RIDGE', 'SARIMA', 'TBATS', 'XGBOOST'], dtype=object)

In [12]:
pivot_df = rmse_df.pivot(index='district', columns='model', values='RMSE')
print(pivot_df)

model                  ANN      ARIMA   BAYESIAN         DT  ELASTICNET  \
district                                                                  
AHMEDNAGAR       22.172191  15.129452  15.812603  20.357996   16.187799   
AKOLA            13.246209   8.658511  11.008634  11.100983   10.188539   
AMRAVATI         20.110462  18.018413  17.324720  21.698651   17.981397   
AURANGABAD             NaN  14.539093  15.362882  36.089026   15.122201   
BEED             13.173882   8.959206   8.098236  14.492909    7.644393   
BHANDARA          3.621990   4.698549   4.649879   4.331396    4.630915   
BULDHANA               NaN   5.461617   8.461511   6.033855    9.847896   
CHANDRAPUR       14.910383  13.504074  14.586615  17.311605   14.263178   
DHULE             9.215523  19.530972  12.637071   8.401389   13.480603   
GADCHIROLI       10.019171  10.804322   9.225120   7.220514    9.361654   
GONDIA            9.066511   7.288154   7.087232   8.046934    7.195237   
HINGOLI           5.55548

In [13]:
pivot_df.to_csv('I40_rmse_pivot.csv', index=True)

In [14]:
# Get the best model for each district (lowest RMSE)
best_models = rmse_df.loc[rmse_df.groupby('district')['RMSE'].idxmin()]

# Display the result
best_models[['district', 'model', 'RMSE']].to_csv('I40_best_models.csv', index=False)